# LLM Optimization Modelling Experiment

In [661]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [674]:
problem = '''You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7'''

## 2. Generate the mathematical model

In [700]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [701]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Water Flow Maximization

**Sets:**

*  **T:** Set of time steps, T = {1, 2, ..., 10}
* **N:** Set of nodes, N = {A, B, C, D, E, F, G}
* **P:** Set of pipes, P = {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG}

**Parameters:**

* **c<sub>ij</sub>:** Capacity of pipe (i, j) ∈ P. Values are given in the problem statement.
* **f(t):** Water inflow function at point A, f(t) = max(-4/30t<sup>3</sup> + t<sup>2</sup> - 0.234t + 3, 0) for t ∈ T.

**Decision Variables:**

* **x<sub>ijt</sub>:** Flow of water through pipe (i, j) ∈ P at time t ∈ T.
* **y<sub>ij</sub>:** Binary variable indicating whether pipe (i, j) ∈ P is activated (y<sub>ij</sub> = 1) or deactivated (y<sub>ij</sub> = 0).
* **r<sub>it</sub>:** Amount of water released at node i ∈ N at time t ∈ T.

**Objective Function:**

Maximize the total water flow out of point G over all time steps:

```
Maximize Z = ∑<sub>t∈T</sub> x<sub>EGt</sub> 
```

**Constraints:**

1. **Flow balance at each node:** For each node i ∈ N and time t ∈ T:
    *  ∑<sub>j:(i,j)∈P</sub> x<sub>ijt</sub> + r<sub>it</sub> = ∑<sub>j:(j,i)∈P</sub> x<sub>jit</sub> + f(t) if i = A 
    *  ∑<sub>j:(i,j)∈P</sub> x<sub>ijt</sub> + r<sub>it</sub> = ∑<sub>j:(j,i)∈P</sub> x<sub>jit</sub> otherwise
2. **Pipe capacity:** For each pipe (i, j) ∈ P and time t ∈ T:
    *  x<sub>ijt</sub> ≤ c<sub>ij</sub> * y<sub>ij</sub>
3. **Pipe activation limit:**
    *  ∑<sub>(i,j)∈P</sub> y<sub>ij</sub> = 10 
4. **Non-negativity:** 
    * x<sub>ijt</sub> ≥ 0 for all (i, j) ∈ P and t ∈ T
    * r<sub>it</sub> ≥ 0 for all i ∈ N and t ∈ T
5. **Binary constraint:**
    * y<sub>ij</sub> ∈ {0, 1} for all (i, j) ∈ P

**Explanation:**

* The objective function maximizes the total water flow out of point G.
* Constraint (1) ensures flow balance at each node, taking into account inflow, outflow, and released water. 
* Constraint (2) limits the flow through each pipe based on its capacity and activation status.
* Constraint (3) ensures that only 10 pipes are activated.
* Constraint (4) enforces non-negativity for flow and release variables.
* Constraint (5) defines the pipe activation variables as binary.

This model can be solved using optimization software to find the optimal pipe activation plan and water flow distribution that maximizes the water output at point G. 


## 3. Generate the pyomo code

In [702]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [703]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample data for pipe capacities (replace with actual data)
c = {
    ('A', 'B'): 5,
    ('A', 'C'): 3,
    ('A', 'F'): 2,
    ('B', 'C'): 4,
    ('B', 'D'): 2,
    ('B', 'E'): 1,
    ('C', 'E'): 3,
    ('C', 'F'): 2,
    ('D', 'E'): 4,
    ('E', 'G'): 6,
    ('F', 'B'): 1,
    ('F', 'G'): 5,
}

# Define sets
T = set(range(1, 11))  # Time steps
N = {'A', 'B', 'C', 'D', 'E', 'F', 'G'}  # Nodes
P = list(c.keys())  # Pipes

# Create model
model = pyo.ConcreteModel()

# Define variables
model.x = pyo.Var(P, T, within=pyo.NonNegativeReals)  # Flow through pipe (i, j) at time t
model.y = pyo.Var(P, within=pyo.Binary)  # Pipe activation
model.r = pyo.Var(N, T, within=pyo.NonNegativeReals)  # Water released at node i at time t

# Define objective function
model.obj = pyo.Objective(expr=sum(model.x['E', 'G', t] for t in T), sense=pyo.maximize)

# Define constraints
model.constraints = pyo.ConstraintList()

# Flow balance constraints
for i in N:
    for t in T:
        if i == 'A':
            model.constraints.add(
                sum(model.x[i, j, t] for j in N if (i, j) in P) + model.r[i, t]
                == sum(model.x[j, i, t] for j in N if (j, i) in P) + max(-4/30*t**3 + t**2 - 0.234*t + 3, 0) 
            )
        else:
            model.constraints.add(
                sum(model.x[i, j, t] for j in N if (i, j) in P) + model.r[i, t]
                == sum(model.x[j, i, t] for j in N if (j, i) in P)
            )

# Pipe capacity constraints
for (i, j) in P:
    for t in T:
        model.constraints.add(model.x[i, j, t] <= c[i, j] * model.y[i, j])

# Pipe activation limit
model.constraints.add(sum(model.y[i, j] for (i, j) in P) == 10)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a solver
solver.solve(model)

# Print results
print("Optimal Solution:")
print("Objective value =", pyo.value(model.obj))

# Print pipe activation status
print("Activated Pipes:")
for (i, j) in P:
    if pyo.value(model.y[i, j]) == 1:
        print(f"Pipe ({i}, {j})")

# Print flow through each pipe at each time step
print("Flow through Pipes:")
for t in T:
    print(f"Time step {t}:")
    for (i, j) in P:
        if pyo.value(model.x[i, j, t]) > 0:
            print(f"  Pipe ({i}, {j}): {pyo.value(model.x[i, j, t])}")
```

This code defines the sets, parameters, variables, objective function, and constraints based on the provided mathematical model. The `c` dictionary is a sample data for pipe capacities and should be replaced with the actual data from the problem statement. The rest of the code directly translates the mathematical model into Pyomo code. The `glpk` solver is used as an example, but other solvers can be used as well. The code then prints the optimal solution, including the objective value, activated pipes, and flow through each pipe at each time step. 


## 4. Input problem data and try running the generated code

In [704]:
import pyomo.environ as pyo

# Sample data for pipe capacities (replace with actual data)
c = {
    ('A', 'B'): 5,
    ('A', 'C'): 3,
    ('A', 'F'): 2,
    ('B', 'C'): 4,
    ('B', 'D'): 2,
    ('B', 'E'): 1,
    ('C', 'E'): 3,
    ('C', 'F'): 2,
    ('D', 'E'): 4,
    ('E', 'G'): 6,
    ('F', 'B'): 1,
    ('F', 'G'): 5,
}

# Define sets
T = set(range(1, 11))  # Time steps
N = {'A', 'B', 'C', 'D', 'E', 'F', 'G'}  # Nodes
P = list(c.keys())  # Pipes

# Create model
model = pyo.ConcreteModel()

# Define variables
model.x = pyo.Var(P, T, within=pyo.NonNegativeReals)  # Flow through pipe (i, j) at time t
model.y = pyo.Var(P, within=pyo.Binary)  # Pipe activation
model.r = pyo.Var(N, T, within=pyo.NonNegativeReals)  # Water released at node i at time t

# Define objective function
model.obj = pyo.Objective(expr=sum(model.x['E', 'G', t] for t in T), sense=pyo.maximize)

# Define constraints
model.constraints = pyo.ConstraintList()

# Flow balance constraints
for i in N:
    for t in T:
        if i == 'A':
            model.constraints.add(
                sum(model.x[i, j, t] for j in N if (i, j) in P) + model.r[i, t]
                == sum(model.x[j, i, t] for j in N if (j, i) in P) + max(-4/30*t**3 + t**2 - 0.234*t + 3, 0) 
            )
        else:
            model.constraints.add(
                sum(model.x[i, j, t] for j in N if (i, j) in P) + model.r[i, t]
                == sum(model.x[j, i, t] for j in N if (j, i) in P)
            )

# Pipe capacity constraints
for (i, j) in P:
    for t in T:
        model.constraints.add(model.x[i, j, t] <= c[i, j] * model.y[i, j])

# Pipe activation limit
model.constraints.add(sum(model.y[i, j] for (i, j) in P) == 10)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a solver
solver.solve(model)

# Print results
print("Optimal Solution:")
print("Objective value =", pyo.value(model.obj))

# Print pipe activation status
print("Activated Pipes:")
for (i, j) in P:
    if pyo.value(model.y[i, j]) == 1:
        print(f"Pipe ({i}, {j})")

# Print flow through each pipe at each time step
print("Flow through Pipes:")
for t in T:
    print(f"Time step {t}:")
    for (i, j) in P:
        if pyo.value(model.x[i, j, t]) > 0:
            print(f"  Pipe ({i}, {j}): {pyo.value(model.x[i, j, t])}")

Optimal Solution:
Objective value = 37.72666666666667
Activated Pipes:
Pipe (A, B)
Pipe (A, C)
Pipe (A, F)
Pipe (B, C)
Pipe (B, D)
Pipe (B, E)
Pipe (C, E)
Pipe (C, F)
Pipe (D, E)
Pipe (E, G)
Flow through Pipes:
Time step 1:
  Pipe (A, B): 1.0
  Pipe (A, C): 2.63266666666667
  Pipe (B, E): 1.0
  Pipe (C, E): 2.63266666666667
  Pipe (E, G): 3.63266666666667
Time step 2:
  Pipe (A, B): 2.46533333333333
  Pipe (A, C): 3.0
  Pipe (B, D): 1.46533333333333
  Pipe (B, E): 1.0
  Pipe (C, E): 3.0
  Pipe (D, E): 1.46533333333333
  Pipe (E, G): 5.46533333333333
Time step 3:
  Pipe (A, B): 3.0
  Pipe (A, C): 3.0
  Pipe (B, D): 2.0
  Pipe (B, E): 1.0
  Pipe (C, E): 3.0
  Pipe (D, E): 2.0
  Pipe (E, G): 6.0
Time step 4:
  Pipe (A, B): 3.0
  Pipe (A, C): 3.0
  Pipe (B, D): 2.0
  Pipe (B, E): 1.0
  Pipe (C, E): 3.0
  Pipe (D, E): 2.0
  Pipe (E, G): 6.0
Time step 5:
  Pipe (A, B): 3.0
  Pipe (A, C): 3.0
  Pipe (B, D): 2.0
  Pipe (B, E): 1.0
  Pipe (C, E): 3.0
  Pipe (D, E): 2.0
  Pipe (E, G): 6.0
Time s

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [706]:
print(response.text)

## Mathematical Optimization Model for Water Flow Maximization

**Sets:**

*  **T:** Set of time steps, T = {1, 2, ..., 10}
* **N:** Set of nodes, N = {A, B, C, D, E, F, G}
* **P:** Set of pipes, P = {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG}

**Parameters:**

* **c<sub>ij</sub>:** Capacity of pipe (i, j) ∈ P. Values are given in the problem statement.
* **f(t):** Water inflow function at point A, f(t) = max(-4/30t<sup>3</sup> + t<sup>2</sup> - 0.234t + 3, 0) for t ∈ T.

**Decision Variables:**

* **x<sub>ijt</sub>:** Flow of water through pipe (i, j) ∈ P at time t ∈ T.
* **y<sub>ij</sub>:** Binary variable indicating whether pipe (i, j) ∈ P is activated (y<sub>ij</sub> = 1) or deactivated (y<sub>ij</sub> = 0).
* **r<sub>it</sub>:** Amount of water released at node i ∈ N at time t ∈ T.

**Objective Function:**

Maximize the total water flow out of point G over all time steps:

```
Maximize Z = ∑<sub>t∈T</sub> x<sub>EGt</sub> 
```

**Constraints:**

1. **Flow balance at each node:

In [707]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample data for pipe capacities (replace with actual data)
c = {
    ('A', 'B'): 5,
    ('A', 'C'): 3,
    ('A', 'F'): 2,
    ('B', 'C'): 4,
    ('B', 'D'): 2,
    ('B', 'E'): 1,
    ('C', 'E'): 3,
    ('C', 'F'): 2,
    ('D', 'E'): 4,
    ('E', 'G'): 6,
    ('F', 'B'): 1,
    ('F', 'G'): 5,
}

# Define sets
T = set(range(1, 11))  # Time steps
N = {'A', 'B', 'C', 'D', 'E', 'F', 'G'}  # Nodes
P = list(c.keys())  # Pipes

# Create model
model = pyo.ConcreteModel()

# Define variables
model.x = pyo.Var(P, T, within=pyo.NonNegativeReals)  # Flow through pipe (i, j) at time t
model.y = pyo.Var(P, within=pyo.Binary)  # Pipe activation
model.r = pyo.Var(N, T, within=pyo.NonNegativeReals)  # Water released at node i at time t

# Define objective function
model.obj = pyo.Objective(expr=sum(model.x['E', 'G', t] for t in T), sense=pyo.maximize)

# Define constraints
model.constraints = pyo.ConstraintList()

# Flow balance constraints
for i in N:
    for t 